In [90]:
import pandas as pd
from commcare_export.checkpoint import CheckpointManagerWithDetails
from commcare_export.commcare_hq_client import CommCareHqClient, AUTH_MODE_APIKEY
from commcare_export.commcare_minilinq import get_paginator, PaginationMode
from datetime import datetime
import re

# CommCare credentials
username = 'amutenha@zimttech.org'  # Replace with your actual CommCare username
domain = 'zdip'  # Your CommCare project domain
hq_host = 'https://zdip.zimttech.org'  # Your CommCare HQ custom host
API_KEY = '92fbb3f98433a6929a33015273ebe7b38b6dac39'


zazicSites= pd.read_excel('zazicFacilities.xlsx')

data = pd.ExcelFile('ZDIPlookup.xlsx')

lookupTable = pd.read_excel(data,'previousVMMCSubmissions')

lookupTable.columns = [col.replace('field: ', '') if col.startswith('field: ') else col for col in lookupTable.columns]

# Initialize API client
api_client = CommCareHqClient(hq_host, domain, username, API_KEY, AUTH_MODE_APIKEY)

# Create a paginator for the 'form' resource (fetching data from a form)
form_paginator = get_paginator(resource='form', pagination_mode=PaginationMode.date_modified, page_size=100)
form_paginator.init()

# Initialize checkpoint manager (None used here for simplicity)
checkpoint_manager = CheckpointManagerWithDetails(None, None, PaginationMode.date_modified)

# Date range for filtering
start_date = datetime(2025, 3, 3)  # 2 December 2024
end_date = datetime(2025, 12,30)  # 6 December 2024

# Specify the app_id to filter forms
desired_app_id ='4e6668dfd6adb19fe71beb316114b48f'  


referrals_data = []
AEs_data = []
general_data = []

# Fetch forms iteratively
# Fetch forms using manual pagination
all_forms = []
offset = 0

params = {
    'limit': 100000,
    'app_id': desired_app_id,
    'received_on_start': start_date.strftime('%Y-%m-%dT%H:%M:%S'),
    'received_on_end': end_date.strftime('%Y-%m-%dT%H:%M:%S')
}

forms = api_client.iterate(
    'form',
    form_paginator,
    checkpoint_manager=checkpoint_manager,
    params=params
)

for form in forms:
    all_forms.append(form)
    if len(all_forms) >= 100000:  # Early stopping if desired
        break

In [91]:
# Print a few fetched forms to verify filtering
for form in all_forms[:5]:  
    print(f"Received On: {form.get('received_on')}, App ID: {form.get('app_id')}")

filteredResults =[]
for i in all_forms:
    if i["app_id"] ==desired_app_id:
        filteredResults.append(i)

print(filteredResults)
# Filter and categorize forms by app_id and date
for form in forms:
    if form.get('app_id') == desired_app_id:
        # print(form)
        form_received_date = datetime.strptime(form['received_on'], "%Y-%m-%dT%H:%M:%S.%fZ")
        if start_date <= form_received_date <= end_date:
            general_data.append(form)
# Convert each list to a pandas DataFrame
general_df = pd.json_normalize(general_data)
general_df.to_excel('test.xlsx')

general_df.columns = [re.split(r'[./]', col)[-1] for col in general_df.columns]


Received On: 2025-08-07T10:47:53.451885Z, App ID: 4e6668dfd6adb19fe71beb316114b48f
Received On: 2025-08-07T10:06:48.503912Z, App ID: 4e6668dfd6adb19fe71beb316114b48f
Received On: 2025-08-07T09:47:19.395370Z, App ID: 4e6668dfd6adb19fe71beb316114b48f
Received On: 2025-08-07T09:01:51.023207Z, App ID: 4e6668dfd6adb19fe71beb316114b48f
Received On: 2025-08-07T08:55:32.329926Z, App ID: 4e6668dfd6adb19fe71beb316114b48f
[{'app_id': '4e6668dfd6adb19fe71beb316114b48f', 'archived': False, 'attachments': {'form.xml': {'content_type': 'text/xml', 'length': 14804, 'url': 'https://zdip.zimttech.org/a/zdip/api/form_attachment/v1/c08d66eb-b0cf-4ec9-8004-1ffd35677a9b/form.xml'}}, 'build_id': '87d44d3ae20a4cee915544aa26bc8328', 'domain': 'zdip', 'edited_by_user_id': None, 'edited_on': None, 'form': {'#type': 'data', '@name': 'ZAZIC VMMC Data Collection Form', '@uiVersion': '1', '@version': '29', '@xmlns': 'http://openrosa.org/formdesigner/2C62E5DB-A039-4F7E-A378-25101817B038', 'client_follow-upGRP': {'fol

In [92]:
# New columns that should NOT be deleted
columns_to_retain = [
    'AERecordingMonth', 'AERecordingSite', 'AERecordingYear', 'aeComments', 'ae_classification', 
    'ae_type_code', 'checkMonthValidation', 'checkYearValidation', 'circumcising_cadre', 'client_age', 
    'date_ae_identified', 'mcMethod', 'vmmc_number', 'followUpTotal', 'recordingMonth', 
    'total_hiv_negative_linked_to_prep', 'total_hiv_positive_linked_to_care', 'ReferrelRecordingMonth', 
    'ReferrelRecordingSite', 'ReferrelRecordingYear', 'reason_for_referral', 'services_referred', 'facilityName', 
    'District', 'recordingMonth', 'year', 'totalAES','adverse_events', 'Site_Name', 
    'totalReferralstoOtherServices','formType','received_on','ageGroupsChange','selectedHTSAgeGroups',
    'fuAgeUpdate' , 'uncircumcised_care',  'uncircumcised_prep','uncircumcised_srh','uncircumcised_sti'
]

# Columns that must be numeric (including the new ones)
numeric_columns = [
    'client_age', 'fu15-19', 'fu20-24', 'fu25-29', 'fu30-34', 'fu40-44', 'fu50',
    'total_hiv_negative_linked_to_prep', 'total_hiv_positive_linked_to_care', 'referralToOtherServices',
    'total_mcs_referred_for_srh_services', 'total_mcs_referred_for_sti_services', 'checkifFacilityHasBeenSaved',
    'hivNegativeNC15-19', 'hivNegativeNC20-24', 'hivNegativeNC25-29', 'hivNegativeNC30-34', 'hivNegativeNC35-39', 
    'hivNegativeNC40-44', 'hivNegativeNC45-49', 'hivNegativeNC50', 'total_surgicalDisposable', 'hivPositive30-34',
    'hivPositiveNC15-19', 'hivPositiveNC20-24', 'hivPositiveNC25-29', 'hivPositiveNC35-39', 'hivPositiveNC40-44',
    'hivPositiveNC45-49', 'hivPositiveNC50', 'total15-19NCHIVTested', 'total20-24NCHIVTested', 'total25-29NCHIVTested',
    'total30-34NCHIVTested', 'total35-39NCHIVTested', 'total40-44NCHIVTested', 'total45-49NCHIVTested', 
    'total50NCHIVTested', 'total_surgicalReusable', 'totalHIVPositiveNC', 'totalhivNegativeNC', 
    'uncircumcisedClientsForHTS', 'hivNegative15-19', 'hivNegative20-24', 'hivNegative25-29', 'hivNegative30-34', 
    'hivNegative40-44', 'hivNegative50', 'total_surgicalDisposable', 'hivPositive15-19', 'hivPositive20-24','hivPositive25-29',
    'hivPositive30-34','hivPositive35-39','hivPositive40-44','hivPositive45-49','hivPositive50',
    'total15-19HIVTested', 'total20-24HIVTested', 'total25-29HIVTested', 'total30-34HIVTested', 'total35-39HIVTested',
    'total40-44HIVTested', 'total45-49HIVTested', 'total50HIVTested', 'total_surgicalReusable', 'totalHIVPositive',
    'totalHIVUntested', 'totalhivNegative', 'TotalMCsBYMethod', 'sgDisposable15-19', 'sgDisposable20-24', 
    'sgDisposable25-29', 'sgDisposable30-34', 'sgDisposable40-44', 'sgDisposable50', 'total_surgicalDisposable',
    'sgReusable15-19', 'sgReusable20-24', 'sgReusable25-29', 'sgReusable30-34', 'sgReusable40-44', 'sgReusable50',
    'total_surgicalReusable', 'total50ByMethod', 'totalShangringMCs', 'totalSurgicalDisposableMCs', 
    'totalSurgicalReusableMCs', 'mc15-19', 'mc20-24', 'mc25-29', 'mc30-34', 'mc35-39', 'mc40-44', 'mc45-49', 
    'mc50', 'totalMCs', 'fu35-39', 'other_referrals_in_detail', 'total_mcs_referred_to_other_services', 
    'hivNegative35-39', 'sgDisposable35-39', 'sgReusable35-39', 'shangring15-19', 'shangring20-24','shangring25-29', 'shangring30-34',
    'shangring35-39', 'shangring40-44','shangring45-49','shangring50', 'fu45-49', 
    'hivNegative45-49', 'sgDisposable45-49', 'sgReusable45-49','AERecordingYear','high_risk_referrals_to_care','LinkagesSelection','copy-1-of-LinkagesSelection',
]

# general_df['AERecordingYear'] = pd.to_numeric(general_df['AERecordingYear'])
# Combine the two lists (retain columns that are in either columns_to_retain or numeric_columns)
columns_to_keep = list(set(columns_to_retain + numeric_columns))

# Identify missing columns
missing_columns = [col for col in columns_to_keep if col not in general_df.columns]

In [93]:
# Add missing columns to the DataFrame and assign zero
for col in missing_columns:
    general_df[col] = 0

# Drop the columns that are not in the columns_to_keep list
# Sort the columns alphabetically in both DataFrames

# For the 'general_df', sort the columns alphabetically
general_df_sorted = general_df[sorted(general_df[columns_to_keep])]

# For the AE-specific columns, we can do the same
ae_columns = [
    'aeComments', 'ae_type_code', 'date_ae_identified', 'AERecordingYear', 'AERecordingSite', 
    'client_age', 'circumcising_cadre', 'ae_classification', 'AERecordingMonth', 'mcMethod','vmmc_number'
]


referrals_columns =['ReferrelRecordingMonth', 
    'ReferrelRecordingSite', 'ReferrelRecordingYear', 'services_referred','reason_for_referral']


other_referrals = []
ae_data_df_sorted = general_df[ae_columns][sorted(ae_columns)]
ae_data_df_sorted = ae_data_df_sorted[ae_data_df_sorted['AERecordingMonth'].notna()]

referralsDF_sorted = general_df[referrals_columns][sorted(referrals_columns)]
referralsDF_sorted = referralsDF_sorted[referralsDF_sorted['ReferrelRecordingMonth'].notna()]


In [94]:
# Function to flatten the nested referral data
def flatten_referral_data(referral_list):
    flattened_referrals = []
    
    # Loop through each referral in the list
    for referral in referral_list:
        flattened_referral = {}
        
        # Flatten the first level of referral data
        for key, value in referral.items():
            if isinstance(value, dict):
                # If the value is a nested dictionary, flatten it
                for nested_key, nested_value in value.items():
                    flattened_referral[f"{key}_{nested_key}"] = nested_value
            else:
                flattened_referral[key] = value
        
        flattened_referrals.append(flattened_referral)
    
    return flattened_referrals



# Function to unpack nested 'question1_question5' field
def unpack_detailedField(df, column):
    if column  in df.columns:
        # Unpack the dictionary inside 'question1_question5' into separate columns
        question_fields = df[column].apply(pd.Series)
        
        # Merge the unpacked fields back into the main DataFrame
        df = pd.concat([df, question_fields], axis=1)
        
        # Drop the original 'question1_question5' column if no longer needed
        df.drop(column, axis=1, inplace=True)
    
    return df



In [95]:
referrals_data = []
singleReferrals = []

for idx, row in general_df.iterrows():
    referral_list = row['other_referrals_in_detail']

    # Case 1: referral list exists and is a valid list
    if isinstance(referral_list, list):
        flattened_referrals = flatten_referral_data(referral_list)

        for ref in flattened_referrals:
            ref['ReferrelRecordingSite'] = row['facilityName']
            ref['District'] = row['District']
            referrals_data.append(ref)

    # Case 2: no referral list, but single referral info is available
    else:
        if pd.notna(row['ReferrelRecordingMonth']):
            singleReferrals.append({
                'ReferrelRecordingMonth': row['ReferrelRecordingMonth'],
                'ReferrelRecordingSite': row['ReferrelRecordingSite'],
                'ReferrelRecordingYear': row['ReferrelRecordingYear'],
                'District': row['District'],
                'reason_for_referral': row['reason_for_referral'],
                'services_referred': row['services_referred']
            })



In [96]:
aes_data =[]
singleAEs = []
for idx, row in general_df.iterrows():
    ae_list = row['adverse_events']
    
    # Check if the value is a list and not NaN
    if isinstance(ae_list, list) and not pd.isna(ae_list).any():  # Use .any() to check the entire list
        flattened_aes = flatten_referral_data(ae_list)
        for ref in flattened_aes:
            ref['District'] = row['District']  # Add the District to each referral record
            aes_data.append(ref)

    else:
        if not pd.isna(row['AERecordingMonth']):
            _object = {
                'AERecordingMonth':row['AERecordingMonth'],
                'AERecordingSite':row['AERecordingSite'],
                'AERecordingYear':row['AERecordingYear'],
                'District':row['District'],
                'aeComments':row['aeComments'],
                'ae_classification':row['ae_classification'],
                'ae_type_code':row['ae_type_code'],
                'circumcising_cadre':row['circumcising_cadre'],
                'client_age':row['client_age'],
                'date_ae_identified':row['date_ae_identified'],
                'mcMethod':row['mcMethod'],
                'vmmc_number':row['vmmc_number']
            }
            singleAEs.append(_object)

In [97]:
# Convert the flattened referrals data into a DataFrame

referrals_df = pd.DataFrame(referrals_data)

# convert the flattened aes data into a df

ae_df = pd.DataFrame(aes_data)

# Unpack the 'question1_question5' field
referrals_df = unpack_detailedField(referrals_df,'question1_question5')
print(f"referrals,{referrals_df}")

referrals_df.to_excel("rfg.xlsx")
ae_df = unpack_detailedField(ae_df,'question3_question1')

ae_df = unpack_detailedField(ae_df,'question1')

ae_df = ae_df.rename(columns={
    'question3_AERecordingMonth': 'AERecordingMonth',
    'question3_AERecordingSite': 'AERecordingSite',
    'question3_AERecordingYear': 'AERecordingYear'
})



if len(singleReferrals)>0:
    tempRefDF = pd.DataFrame(singleReferrals)
    referrals_df = pd.concat([referrals_df,tempRefDF])

if len(singleAEs)>0:
    tempRefDF = pd.DataFrame(singleAEs)
    ae_df = pd.concat([ae_df,tempRefDF])


if not referrals_df.empty:
    try:
        referrals_df = referrals_df.drop(columns = ['question1_cancelthisReferralReport',0]).drop_duplicates()
    except:
        pass

if not ae_df.empty:
    try:
        ae_df = ae_df.drop(columns = ['checkMonthValidation','checkYearValidation']).drop_duplicates()
    except:
        pass

general_df_sorted = general_df_sorted.drop(columns =['adverse_events','other_referrals_in_detail']).drop_duplicates()


zazicSites_unique = zazicSites.drop_duplicates(subset='facilityName')


general_df_sorted = general_df_sorted.loc[:, ~general_df_sorted.columns.duplicated(keep='last')]

# Merge to bring in facilityType from zazicSites_unique
general_df_sorted = general_df_sorted.merge(zazicSites_unique[['facilityName', 'facilityType']], 
                                            on='facilityName', 
                                            how='left')

# general_df_sorted['received_on'] = pd.to_datetime(general_df_sorted['received_on']).dt.tz_localize(None)

# Fill missing values with 0
general_df_sorted = general_df_sorted.fillna(0)

# Save the original cleaned DataFrame
general_df_sorted.to_excel("generalDF.xlsx", index=False)

# Remove duplicates where formType is 'submission'
df_filtered = general_df_sorted[~(
    (general_df_sorted.duplicated(subset=['facilityName', 'recordingMonth', 'year'], keep=False)) &
    (general_df_sorted['formType'] == 'submission') &
    (general_df_sorted['facilityName'] != 'Beula')
)]

df_filtered = df_filtered[~(
    (df_filtered['ae_classification'] == 0) & 
    (df_filtered['facilityName'] == 'Beula')
)]

# Keep the latest entry per group based on 'received_on'
general_df_sorted = df_filtered.loc[
    df_filtered.groupby(['facilityName', 'recordingMonth', 'year'])['received_on'].idxmax()
]


# Sort the final DataFrame for readability
general_df_sorted = general_df_sorted.sort_values(by=['facilityName', 'year', 'recordingMonth'])

# Save the final filtered and sorted DataFrame
general_df_sorted.to_excel("generalDF1.xlsx", index=False)


# Step 1: Identify the latest 'correction' row for each facilityName, recordingMonth, and year
latest_correction_idx = general_df_sorted[general_df_sorted['formType'] == 'correction'].groupby(['facilityName', 'recordingMonth', 'year'])['received_on'].idxmax()

# Step 2: Create a dictionary to store the latest correction rows
latest_corrections = general_df_sorted.loc[latest_correction_idx]
# Step 3: Iterate through each submission row and check if there is a corresponding correction row
for idx, row in general_df_sorted[general_df_sorted['formType'] == 'submission'].iterrows():
    # Get the corresponding correction row for the current submission
    correction_row = latest_corrections[
        (latest_corrections['facilityName'] == row['facilityName']) &
        (latest_corrections['recordingMonth'] == row['recordingMonth']) &
        (latest_corrections['year'] == row['year'])
    ]
    
# Step 1: Identify the latest 'correction' row for each facilityName, recordingMonth, and year
latest_correction_idx = general_df_sorted[general_df_sorted['formType'] == 'correction'].groupby(['facilityName', 'recordingMonth', 'year'])['received_on'].idxmax()

# Step 2: Create a dictionary to store the latest correction rows
latest_corrections = general_df_sorted.loc[latest_correction_idx]

# Define the age groups you're interested in
age_groups = ['15-19_yrs', '20-24_yrs', '25-29_yrs', '30-34_yrs', '35-39_yrs', '40-44_yrs', '45-49_yrs', '50+_yrs']

# Columns to update for each age group
columns_to_update = ['mc', 'sgDisposable', 'sgReusable', 'shangring', 'fu', 'hivPositive', 'hivNegative']

# DataFrame to store the final results
final_df = general_df_sorted.copy()

# Step 1: Iterate through submission rows
for idx, row in general_df_sorted[general_df_sorted['formType'] == 'submission'].iterrows():
    facility, month, year = row['facilityName'], row['recordingMonth'], row['year']

    # Step 2: Find corresponding correction row
    correction_row = latest_corrections[
        (latest_corrections['facilityName'] == facility) &
        (latest_corrections['recordingMonth'] == month) &
        (latest_corrections['year'] == year)
    ]

    print(correction_row)
    if not correction_row.empty:  # If a correction row exists
        # Step 3: Check if a row exists in lookupTable

        lookup_match = lookupTable[
            (lookupTable['facilityName'] == facility) &
            (lookupTable['recordingMonth'] == month) &
            (lookupTable['year'] == year)
        ]

        print(lookup_match)

        if not lookup_match.empty:
            # Apply corrections to lookupTable row instead of submission row
            target_idx = lookup_match.index[0]  # Get index of matched row
            target_df = lookupTable  # Apply updates to lookupTable
            
            # Step 4: Update the matched row in lookupTable
            age_groups_change = correction_row['ageGroupsChange'].values[0]
            for age_group in age_groups:
                if age_group in age_groups_change:
                    for column_prefix in columns_to_update:
                        column_name = f'{column_prefix}{age_group.split("_")[0]}'
                        target_df.at[target_idx, column_name] = correction_row[column_name].values[0]

            # NEW LOGIC FOR 'total_mcs_referred_for_srh_services' UPDATE (LOOKUPTABLE PATH)
            if 'copy-1-of-LinkagesSelection' in correction_row.columns and 'referral_srh' in correction_row['copy-1-of-LinkagesSelection'].values[0]:
                if 'total_mcs_referred_for_srh_services' in correction_row.columns:
                    target_df.at[target_idx, 'total_mcs_referred_for_srh_services'] = correction_row['total_mcs_referred_for_srh_services'].values[0]

            if 'copy-1-of-LinkagesSelection' in correction_row.columns and 'referral_sti' in correction_row['copy-1-of-LinkagesSelection'].values[0]:
                if 'total_mcs_referred_for_sti_services' in correction_row.columns:
                    target_df.at[target_idx, 'total_mcs_referred_for_sti_services'] = correction_row['total_mcs_referred_for_sti_services'].values[0]


            # Remove the submission row from final dataframe
            final_df.drop(index=idx, inplace=True)

            # Add the updated row from lookupTable to final dataframe
            final_df = pd.concat([final_df, lookupTable.loc[[target_idx]]], ignore_index=True)

        else:
            # No match in lookupTable, apply changes to the original submission row
            print("working here")

            age_groups_change = correction_row['ageGroupsChange'].values[0]
            for age_group in age_groups:
                if age_group in age_groups_change:
                    for column_prefix in columns_to_update:
                        column_name = f'{column_prefix}{age_group.split("_")[0]}'
                        final_df.at[idx, column_name] = correction_row[column_name].values[0]

            # NEW LOGIC FOR 'total_mcs_referred_for_srh_services' UPDATE (ORIGINAL SUBMISSION PATH)
            if 'copy-1-of-LinkagesSelection' in correction_row.columns and 'referral_srh' in correction_row['copy-1-of-LinkagesSelection'].values[0]:
                if 'total_mcs_referred_for_srh_services' in correction_row.columns:
                    final_df.at[idx, 'total_mcs_referred_for_srh_services'] = correction_row['total_mcs_referred_for_srh_services'].values[0] 


            if 'copy-1-of-LinkagesSelection' in correction_row.columns and 'referral_sti' in correction_row['copy-1-of-LinkagesSelection'].values[0]:
                if 'total_mcs_referred_for_sti_services' in correction_row.columns:
                    final_df.at[target_idx, 'total_mcs_referred_for_sti_services'] = correction_row['total_mcs_referred_for_sti_services'].values[0]






# Step 5: Remove correction rows from final dataframe
final_df = final_df[~final_df.index.isin(latest_corrections.index)]
final_df = final_df[final_df['formType']!='correction']

# Reset index to maintain consistency
final_df = general_df_sorted
final_df.reset_index(drop=True, inplace=True)


correction_df = final_df[final_df['formType'] == 'correction']

# Find the most recent 'correction' row per group
# You can change 'ageGroupsChange' to another column if it's not a datetime or relevant
latest_corrections = correction_df.drop_duplicates(subset=['facilityName', 'recordingMonth', 'year'])

# Resulting DataFrame of the most recent correction per group
latest_corrections.reset_index(drop=True, inplace=True)

latest_corrections.to_excel("corrections.xlsx")

final_df = final_df[~final_df["facilityName"].isin(["Masikati", "Murambi"])]

# Save final result
# zazicDF_updated.to_excel("data.xlsx", index=False)
# Now save the DataFrame to Excel
with pd.ExcelWriter('data.xlsx', engine='xlsxwriter') as writer:
    # Save the sorted general data to the first sheet
    final_df.to_excel(writer, sheet_name='Statistics', index=False)
    
    # Save the sorted AE data to a new sheet
    ae_df.drop_duplicates().to_excel(writer, sheet_name='AEs', index=False)
    
    # Save the flattened referrals data to a new sheet
    referrals_df.to_excel(writer, sheet_name='Referrals', index=False)

referrals,Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: [AERecordingMonth, AERecordingSite, AERecordingYear, District, LinkagesSelection, ReferrelRecordingMonth, ReferrelRecordingSite, ReferrelRecordingYear, Site_Name, TotalMCsBYMethod, aeComments, ae_classification, ae_type_code, ageGroupsChange, checkMonthValidation, checkYearValidation, checkifFacilityHasBeenSaved, circumcising_cadre, client_age, copy-1-of-LinkagesSelection, date_ae_identified, facilityName, followUpTotal, formType, fu15-19, fu20-24, fu25-29, fu30-34, fu35-39, fu40-44, fu45-49, fu50, fuAgeUpdate, high_risk_referrals_to_care, hivNegative15-19, hivNegative20-24, hivNegative25-29, hivNegative30-34, hivNegative35-39, hivNegative40-44, hivNegative45-49, hivNegative50, hivNegativeNC15-19, hivNegativeNC20-24, hivNegativeNC25-29, hivNegativeNC30-34, hivNegativeNC35-39, hivNegativeNC40-44, hivNegativeNC45-49, hivNegativeNC50, hivPositive15-19, hivPositive20-24, hivPositive25-29, hivPositive30-34, hivPositive

In [98]:
referrals_df

,ReferrelRecordingMonth,ReferrelRecordingSite,ReferrelRecordingYear,District,reason_for_referral,services_referred
0,May,Tengwe Clinic,2025,HURUNGWE,Malaria,Clinic
1,April,Kushinga Clinic,2025,MARONDERA,NaN,NaN
2,February,Turf Clinic,2025,MHONDORO,NaN,NaN
3,December,Donain,2024,MHONDORO,NaN,NaN
4,March,Chinhere clinic,2025,HURUNGWE,Viral Load,Client on ART referred for viral load
